<a href="https://colab.research.google.com/github/jessfeld/netpyne_course_2021/blob/main/import_cells.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## [Importing Cells in NetPyNE](http://www.netpyne.org/advanced.html#importing-externally-defined-cell-models)

# (1) Clone repository and compile mod files

**Determine your location in the directory structure**

In [ ]:
!pwd

**Move to (or stay in) the '/content' directory**

In [ ]:
cd /content/

**Ensure you are in the correct directory** --> *Expected output: "/content"*

In [ ]:
!pwd

**Install NEURON and NetPyNE, and import matplotlib**

In [ ]:
!pip install neuron
!pip install netpyne 
import matplotlib
import os
import json

In [ ]:
%matplotlib inline

This next line will **detect if the directory already exists** (i.e. you are re-running this code), and will **delete it** to prevent future errors. 

In [ ]:
if os.path.isdir('/content/cells_netpyne2021'):
  !rm -r /content/cells_netpyne2021

**Clone repository with the necessary cell and mod files**

In [ ]:
!git clone https://github.com/ericaygriffith/cells_netpyne2021.git

**Move into the repository with all the necessary files**

In [ ]:
cd cells_netpyne2021/

**Ensure you are in the repository with the 'pwd' command** --> *Expected output*: '/content/cells_netpyne2021'

In [ ]:
!pwd

**Compile the mod files** --> *Expected output:* creation of an 'x86_64' directory 

In [ ]:
!nrnivmodl

# (2) Importing cells from different file formats

**Set up netParams object**

In [ ]:
from netpyne import specs, sim   

# Network parameters
netParams = specs.NetParams()  # object of class NetParams to store the network parameters

**2a. Import cell from *.json* format**

In [ ]:
netParams.loadCellParamsRule(label='TC_reduced', fileName = 'TC_reduced_cellParams.json')

In [ ]:
netParams.cellParams['TC_reduced']

**2b. Import a detailed morphology from a *.swc* file**

In [ ]:
netParams.importCellParams(
        label='PYR_HH3D_swc',
        conds={'cellType': 'PYR', 'cellModel': 'HH3D_swc'},
        fileName='BS0284.swc',
        cellName='swc_cell')

In [ ]:
netParams.cellParams.keys()

**2c. Import a cell from a *.hoc* (NEURON) file**

In [ ]:
netParams.importCellParams(
        label='PYR_HH3D_hoc',
        conds={'cellType': 'PYR', 'cellModel': 'HH3D_hoc'},
        fileName='geom.hoc',
        cellName='E21',
        importSynMechs=False)

In [ ]:
netParams.cellParams.keys()

**2d. Import a cell from a *.py* (python) file**

In [ ]:
netParams.importCellParams(
        label='sRE_py',
        conds={'cellType': 'sRE', 'cellModel': 'HH'},
        fileName='sRE.py',
        cellName='sRE',
        importSynMechs=False)

In [ ]:
netParams.cellParams.keys()

**EXERCISE: import the other swc file contained in the cells_netpyne2021 directory**

In [ ]:
netParams.importCellParams(
    label='mouse_hipp_swc',
    conds={'cellType': 'hipp','cellModel': 'HH3D'},
    fileName='mouseGABA_hipp.swc',
    cellName='swc_hippCell'
)

In [ ]:
netParams.cellParams.keys()

# (3) Explore and manipulate cell parameters

**Explore the cell types located in the netParams.cellParams dictionary**

In [ ]:
netParams.cellParams.keys()

**EXERCISE: Find the geometry (length & diameter) of the soma compartment for each of the above cells**

**EXERCISE: List all of the channel mechanisms in the soma compartment of the thalamocortical cell model (TC_reduced)**

**Now we want to explore (and change) the values of a channel parameter in a given cell model**

In [ ]:
netParams.cellParams['TC_reduced']['secs']['soma']['mechs'].keys()

In [ ]:
netParams.cellParams['TC_reduced']['secs']['soma']['mechs']['pas'].keys()

In [ ]:
netParams.cellParams['TC_reduced']['secs']['soma']['mechs']['pas']['g']

**EXERCISE: Change the conductance of the leak channel in the soma compartment of the reticular cell model (sRE.py)**



**EXERCISE: Insert a passive leak channel ('pas') into the soma compartment of the mouseGABA_hipp.swc cell model**


**EXERCISE: Change the capacitance of all compartments in the model defined by BS0284.hoc (PYR_HH3D_swc)**

**Now let's see how these changes affect the cell behavior by plotting cell's response to current input before and after param changes!**

**EXERCISE: First create a population of thalamocortical cells**

**EXERCISE: Add hyperpolarizing current clamp stimulation of -0.1 nA to thalamocortical cell pop** 

**Add cfg params**

In [ ]:
## cfg  
cfg = specs.SimConfig()					            # object of class SimConfig to store simulation configuration
cfg.duration = 2*1e3 						            # Duration of the simulation, in ms
cfg.dt = 0.01								                # Internal integration timestep to use
cfg.verbose = 1							                # Show detailed messages 
cfg.recordTraces = {'V_soma':{'sec':'soma','loc':0.5,'var':'v'}}  # Dict with traces to record
cfg.recordStep = 0.01 			
cfg.filename = 'model_output'  			# Set file output name
cfg.saveJson = False
cfg.analysis['plotTraces'] = {'include': [0], 'saveFig': True} # Plot recorded traces for this list of cells
cfg.hParams['celsius'] = 36


**Create network and run simulation**

In [ ]:
sim.createSimulateAnalyze(netParams = netParams, simConfig = cfg)

**EXERCISE: We see a rebound burst! T-type calcium channels are normally considered responsible for this behavior. What happens if we set the conductance of this channel to 0?**

**cfg params**

In [ ]:
## cfg  
cfg = specs.SimConfig()					            # object of class SimConfig to store simulation configuration
cfg.duration = 2*1e3 						            # Duration of the simulation, in ms
cfg.dt = 0.01								                # Internal integration timestep to use
cfg.verbose = 1							                # Show detailed messages 
cfg.recordTraces = {'V_soma':{'sec':'soma','loc':0.5,'var':'v'}}  # Dict with traces to record
cfg.recordStep = 0.01 			
cfg.filename = 'model_output'  			# Set file output name
cfg.saveJson = False
cfg.analysis['plotTraces'] = {'include': [0], 'saveFig': True} # Plot recorded traces for this list of cells
cfg.hParams['celsius'] = 36

**Run the sim**

In [ ]:
sim.createSimulateAnalyze(netParams = netParams, simConfig = cfg)

# (4) Plotting Morphology

In [ ]:
netParams.popParams['HH3D_pop_hoc'] = {'cellType': 'PYR', 'numCells': 1, 'cellModel': 'HH3D_hoc'}


In [ ]:
sim.createSimulateAnalyze(netParams = netParams, simConfig = cfg)

In [ ]:
sim.analysis.plotShape(includePre = [], includePost=['HH3D_pop_hoc'], showSyns=False, figSize=(4,9), dist=0.8, saveFig=True)

**EXERCISE: Try plotting the morphology of other cell models**

# (5) Making a Network

**EXERCISE: To begin creating a network, specify the geometry of the area you would like to model.**

In [ ]:
netParams.sizeX = 200

**Now let's set the propagation velocity and length constant:**

In [ ]:
netParams.propVelocity = 100.0 # propagation velocity (um/ms)

In [ ]:
netParams.probLengthConst = 150.0 # length constant for conn probability (um)

**EXERCISE: Now establish a few populations of cells**

**Now we need some synaptic mechanism parameters**

In [ ]:
netParams.synMechParams['exc'] = {'mod': 'Exp2Syn', 'tau1': 0.8, 'tau2': 5.3, 'e': 0} # NMDA synaptic mechanism

In [ ]:
netParams.synMechParams['inh'] = {'mod': 'Exp2Syn', 'tau1': 0.6, 'tau2': 8.5, 'e': -75}  # GABA synaptic mechanism


**Add some network stimulation parameters**

In [ ]:
netParams.stimSourceParams['bkg'] = {'type': 'NetStim', 'rate': 40, 'noise': 0.3}

**EXERCISE: modify the line below such that your stim object can target the populations in your network**

In [ ]:
netParams.stimTargetParams['bkg->all'] = {'source': 'bkg', 'conds': {'cellType': ['E','I']}, 'weight': 10.0, 'sec': 'soma', 'delay': 'max(1, normal(5,2))', 'synMech': 'exc'}


**Add cell connectivity rules**

**EXERCISE: modify the lines below to fit your network**

In [ ]:
netParams.connParams['E->all'] = {
  'preConds': {'cellType': 'E'}, 'postConds': {'y': [100,1000]},  #  E -> all (100-1000 um)
  'probability': 0.1 ,                  # probability of connection
  'weight': '5.0*post_ynorm',         # synaptic weight
  'delay': 'dist_3D/propVelocity',      # transmission delay (ms)
  'synMech': 'exc'}                     # synaptic mechanism

**EXERCISE: Add the appropriate line(s) to run the network and plot a 2D representation of your network w/ connectivity between cells**

In [ ]:
cfg.analysis['plot2Dnet'] = {'saveFig': True}                                                # plot 2D cell positions and connections
cfg.analysis['plotConn'] = {'saveFig': True}                                                 # plot connectivity matrix
